In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

#for the cartesian product
import itertools

#to smooth the MFPADs
import scipy as sp
import scipy.ndimage

#to create the mesh
import triangulation as tr
from scipy.spatial import Delaunay
from pyntcloud import PyntCloud, structures

import plotly.graph_objects as go

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

#low old binning (wrong?) (12,6)
# fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")


#high new binning (24,12)
fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

k=fileRCR[loc].keys()
d=dict(fileRCR[loc].classes()) # shows the type of each item
dict(fileRCR[loc]) # keys : values
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
# type(check)
# check._members()

In [ ]:
def getamesh(x,y,z,d):
    temp = pd.DataFrame(np.hstack((x[:,None], y[:,None], z[:,None])))
    temp.columns = ["x", "y", "z"]
    sph = PyntCloud(temp)
    x,  y,  z = sph.points["x"], sph.points["y"], sph.points["z"]
    pts = sph.points[['x', 'y', 'z']]
    delaun = structures.Delaunay3D(pts)
    
    delaun.compute()
    mesh = delaun.get_mesh()
    tri = mesh[['v1', 'v2', 'v3']].values
    I, J, K = tri.T
    return x,y,z,I,J,K

def makeamesh (x,y,z,d):
    points2d_trace=go.Scatter(x=x, y=y, mode='markers', marker_color='red', marker_size=6)
    point_trace=go.Scatter(x=x, y=y, 
                         mode='markers',
                         name='points',
                         marker_color='red',
                         marker_size=6)
    pts2d=np.array([x,y]).T
    tri=Delaunay(pts2d)
    delaunay_tri = tr.triangulation_edges(pts2d, tri.simplices, linewidth=1)

    i, j, k = tri.simplices.T
    my_mesh3d = go.Mesh3d(
                        x = x,
                        y = y,
                        z = z,
                        i=i, j=j, k=k,
                        colorscale='deep_r',
                        colorbar_thickness=25,
                        intensity=d, 
                        flatshading=True)
    
    points3d=np.array([x,y,z]).T
    delaun_tri3d=tr.triangulation_edges(points3d, tri.simplices)
    return delaunay_tri, point_trace, my_mesh3d, delaun_tri3d

def shift_func(a, flag=0.):
    if a.shape[0]>12:
        c=a.reshape(12,6)
        flag=1.
        if c[0][0] == c[0][1]:
            b=(c[1][0]-c[0][0])/2
            a=np.add(c,b)
        else:
            b=(c[0][1]-c[0][0])/2
            a=np.add(c,b)
    else:
        if a[0][0] == a[0][1]:
            b=(a[1][0]-a[0][0])/2
            a=np.add(a,b)
        else:
            b=(a[0][1]-a[0][0])/2
            a=np.add(a,b)
    if flag == 1.:
        # a=np.ravel(a)
        a=a.reshape(-1)
    return a

def cosphi_func(key, cosphi):
    ctheta_nc = float((str(key).split("costheta_")[1]).split("_phi")[0])
    phi_nc = float((str(key).split("phi_")[1]).split(";")[0])
    # alternative method with import re
    #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1)
    cosphi.append((ctheta_nc, phi_nc))
    return cosphi

def import_MFPAD(file, MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred, run_MFPAD=0., run_cos=0.):
    """
    Loads the MFPADs and the cos(theta) from the .root files.
    6 inputs + a parameter
    NOTE: MFPAD_xy and ctheta_c have originally +1 dimensions compare to the z values.
    For the sake of iminiut, cos(theta) is centered on the middle of the bins.
    """
    for key, value in file[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            cosphi=cosphi_func(key,cosphi)
            temp=np.array(file[filename].numpy())
            MFPAD.append(temp[0]) # it is a list!
            if run_MFPAD == 0.:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_MFPAD=1.
        elif "cos(theta)" in filename.lower():
            temp=np.array(file[filename].numpy())
            ctheta.append(temp[0]) # it is a list!
            if run_cos == 0.:
                ctheta_c.append(temp[1])
                ctheta_cred.append(np.array((ctheta_c[0][1:] + ctheta_c[0][:-1])/2)) #! reduced of 1 dimension
                run_cos=1.
    return MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred

In [ ]:
# initilialises a list
MFPAD_RCR=[]; ctheta_RCR=[]; ctheta_xaxis_RCR=[]; cosphi_RCR=[];
MFPAD_RCL=[]; ctheta_RCL=[]; ctheta_xaxis_RCL=[]; cosphi_RCL=[];
MFPAD_SCR=[]; ctheta_SCR=[]; ctheta_xaxis_SCR=[]; cosphi_SCR=[];
MFPAD_SCL=[]; ctheta_SCL=[]; ctheta_xaxis_SCL=[]; cosphi_SCL=[];

MFPAD_xy=[];
ctheta_cred=[];

import_MFPAD(fileRCR, MFPAD_RCR, cosphi_RCR, MFPAD_xy, ctheta_RCR, ctheta_xaxis_RCR, ctheta_cred)
import_MFPAD(fileRCL, MFPAD_RCL, cosphi_RCL, MFPAD_xy, ctheta_RCL, ctheta_xaxis_RCL, ctheta_cred)
import_MFPAD(fileSCR, MFPAD_SCR, cosphi_SCR, MFPAD_xy, ctheta_SCR, ctheta_xaxis_SCR, ctheta_cred)
import_MFPAD(fileSCL, MFPAD_SCL, cosphi_SCL, MFPAD_xy, ctheta_SCL, ctheta_xaxis_SCL, ctheta_cred)

# converting into nparray
MFPAD_RCR=np.array(MFPAD_RCR); ctheta_RCR=np.array(ctheta_RCR); ctheta_xaxis_RCR=np.array(ctheta_xaxis_RCR[0]); cosphi_RCR=np.array(cosphi_RCR);
MFPAD_RCL=np.array(MFPAD_RCL); ctheta_RCL=np.array(ctheta_RCL); ctheta_xaxis_RCL=np.array(ctheta_xaxis_RCL[0]); cosphi_RCL=np.array(cosphi_RCL);
MFPAD_SCR=np.array(MFPAD_SCR); ctheta_SCR=np.array(ctheta_SCR); ctheta_xaxis_SCR=np.array(ctheta_xaxis_SCR[0]); cosphi_SCR=np.array(cosphi_SCR);
MFPAD_SCL=np.array(MFPAD_SCL); ctheta_SCL=np.array(ctheta_SCL); ctheta_xaxis_SCL=np.array(ctheta_xaxis_SCL[0]); cosphi_SCL=np.array(cosphi_SCL);

ctheta_cred=ctheta_cred[0];
cosphi=cosphi_RCR;

In [ ]:
#FOR LIGHT DIRECTION, 72 each
cosn=np.array([col[0] for col in cosphi]);
phin=np.array([col[1] for col in cosphi]);
#FOR LIGHT DIRECTION

#FOR MFPDAS from here onwards
#cosm (13,) phim (25,)
cosm=MFPAD_xy[0][1]
phim=MFPAD_xy[0][0]

#cosM (12,) phiM (24,)
cosM=(MFPAD_xy[0][1][1:] + MFPAD_xy[0][1][:-1])/2
phiM=(MFPAD_xy[0][0][1:] + MFPAD_xy[0][0][:-1])/2

#effective dimensions to avoid retyping
c=cosM.shape[0]
p=phiM.shape[0]

#cartesian product of unique values, (288,2) each
phicos_MFPAD = np.array(list(itertools.product(phiM, cosM)))
cosphi_MFPAD = np.array(list(itertools.product(cosM, phiM)))
# print(phicos_MFPAD,cosphi_MFPAD)

#(288,1) for each
ctheta_adm = np.array([col[0] for col in cosphi_MFPAD])
theta_rad = np.arccos(ctheta_adm)
phi_ang = np.array([col[1] for col in cosphi_MFPAD])
phi_rad = phi_ang * np.pi/180.

In [ ]:
#look to the values in b1 map and found the MFPAD
cosphi_sh=list(zip(shift_func(cosphi[:,0]),shift_func(cosphi[:,1])))
df=pd.DataFrame(cosphi_sh, columns=["ctheta","phi"])
df.loc[df["phi"] == -165]

In [ ]:
#insert hier the corrispondent index number coming from the comparison with the b1 map values
checkthis=31

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20, 6.5),sharex='col')
fig.subplots_adjust(hspace = .5, wspace=.5)

ax[0].contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),MFPAD_RCL[checkthis].T)
ax[1].contourf(phiM,cosM,MFPAD_RCL[checkthis].T) #has to be transposed becasue how it has been defined in lmf2root
ax[2].contourf(cosM,phiM,MFPAD_RCL[checkthis]) #this is the original x=ctheta, y=phi

fig.tight_layout()

MFPAD_RCL[0].shape

In [ ]:
#as specified later, sigma phi is double sigma x
plt.contourf(phi_ang.reshape(c,p),ctheta_adm.reshape(c,p),smoothgauss(MFPAD_RCL[checkthis],0.3,0.6).T)
plt.legend(loc="best")

In [ ]:
#from spherical to cartesian coordinates

counts=MFPAD_RCR[checkthis].reshape(-1)
x = counts * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(p,c)
y = counts * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(p,c)
z = counts * np.cos(theta_rad)
Z = z.reshape(p,c)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

In [ ]:
#from spherical to cartesian coordinates
#SMOOTHED by sigmax (ctheta) and sigmay (phi)
# sigma phi (y) can be double the value of x 

counts=smoothgauss(MFPAD_RCR[checkthis],0.3,0.6).reshape(-1)
xs = counts * np.sin(theta_rad) * np.cos(phi_rad)
Xs = xs.reshape(p,c)
ys = counts * np.sin(theta_rad) * np.sin(phi_rad)
Ys = ys.reshape(p,c)
zs = counts * np.cos(theta_rad)
Zs = zs.reshape(p,c)

ds = np.sqrt(xs**2+ys**2+zs**2)
d_matrixs = np.sqrt(Xs**2+Ys**2+Zs**2)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                    marker=dict( size=5,color=d, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

fig.update_layout(title="TFMeOx MFPADs EXP", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
fig.show()

figs = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs, mode='markers',
                    marker=dict( size=5,color=ds, colorscale='Viridis', opacity=1., showscale=True)
                    # color set colours to an array/list of desired values  
                    # show scale to show the legend according to the color
                    )])

figs.update_layout(title="TFMeOx MFPADs EXP smoothed", width=500, height=500, margin=dict(l=65, r=50, b=65, t=90)) #margin=dict(l=0, r=0, b=0, t=0))
figs.show()

In [ ]:
delaunay_tri, point_trace, my_mesh3d, delaun_tri3d = makeamesh(xs,ys,zs,ds)

fig2 = go.Figure(data=[delaunay_tri, point_trace])
fig2.update_layout(width=500, height=500);
fig2.show()

my_layout = dict(width=800, height=600,
                 scene_camera_eye=dict(x=1.5, y=1.5, z=1), 
                 scene_aspectratio=dict(x=1.5, y=1.5, z=1))

fig3d = go.Figure(data=[my_mesh3d, delaun_tri3d])

fig3d.show()

In [ ]:
x1,y1,z1,I1,J1,K1 = getamesh(xs,ys,zs,ds)

fig1 = go.Figure(go.Mesh3d(x=x1, y=y1, z=z1, 
                           i=I1, j=J1, k=K1, 
                           intensity=d,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig1.update_layout(width=800, height=800)
fig1.show()


In [ ]:
figsurf = go.Figure(data=[go.Surface(z=Z, surfacecolor=d_matrix)])
figsurf.update_layout(title='TFMeOx MFPADs EXP surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
figsurf.show()

figsurfs = go.Figure(data=[go.Surface(z=Zs, surfacecolor=d_matrixs)])
figsurfs.update_layout(title='TFMeOx MFPADs EXP surf SMOOTHED', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
figsurfs.show()


In [ ]:
# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig, axes = plt.subplots(6,12, figsize=(25, 12), sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
for i,ax in zip(range(72),axes.flatten()) :
    ax.contourf(phiM,cosM,MFPAD_RCL[i].T)
    
fig.tight_layout()
plt.show()